In [ ]:
from pathlib import Path
import os
if not str(Path.cwd()).endswith('/Cloud-Telescope'):
    os.chdir(Path.cwd().parent)
    print(f'New working dir is {Path.cwd()}')    
try:
    import dns
except ModuleNotFoundError:
    import sys
    !{sys.executable} -m pip install dnspython
    import dns
try:
    import brokenaxes
except ModuleNotFoundError:
    import sys
    import subprocess
    subprocess.check_call([sys.executable, "-m", "pip", "install", "brokenaxes"])
    import brokenaxes
import importlib
import pandas as pd

import src.d01_data.read_files  
importlib.reload(src.d01_data.read_files)
import src.d01_data.save_files
importlib.reload(src.d01_data.save_files)
import src.d01_data.log_names_enum
importlib.reload(src.d01_data.log_names_enum)

import src.d04_analysation.count_percent
importlib.reload(src.d04_analysation.count_percent)

import src.d02_intermediate.filter_telescope_ips
importlib.reload(src.d02_intermediate.filter_telescope_ips)

import src.d04_analysation.scanning
importlib.reload(src.d04_analysation.scanning)

import src.d04_analysation.arrival_rate
importlib.reload(src.d04_analysation.arrival_rate)

import src.d04_analysation.scnas_mapped_to_country
importlib.reload(src.d04_analysation.scnas_mapped_to_country)

import src.d04_analysation.scans_proto_country
importlib.reload(src.d04_analysation.scans_proto_country)

import src.d04_analysation.find_scanner
importlib.reload(src.d04_analysation.find_scanner)

import src.d04_analysation.scan_aggrssivity_mapped_countries
importlib.reload(src.d04_analysation.scan_aggrssivity_mapped_countries)

import src.d04_analysation.reverse_dns_lookup
importlib.reload(src.d04_analysation.reverse_dns_lookup)

import src.d04_analysation.scan_education
importlib.reload(src.d04_analysation.scan_education)

import src.d03_visiualisation.tabel_visiualisation
importlib.reload(src.d03_visiualisation.tabel_visiualisation)

import src.d03_visiualisation.bar_diagram
importlib.reload(src.d03_visiualisation.bar_diagram)

import src.d03_visiualisation.head_map
importlib.reload(src.d03_visiualisation.head_map)

import src.d03_visiualisation.graf_arrival_rate
importlib.reload(src.d03_visiualisation.graf_arrival_rate)

import src.d03_visiualisation.graf_interval_threshold
importlib.reload(src.d03_visiualisation.graf_interval_threshold)

import src.d03_visiualisation.tabel_visiualisation
importlib.reload(src.d03_visiualisation.tabel_visiualisation)

### load final scanner

In [ ]:
import pandas as pd
from src.d01_data.read_files import *
from src.d04_analysation.scanning import *
from src.config import *

port_scan: pd.DataFrame = pd.read_feather(f'/home/tobias/work/Cloud-Telescope/data/scanner/port_scanner/p_threshold6interval420')
port_scan = port_scan.sort_values(by='ts', ascending=True)
port_scan = port_scan.reset_index(drop=True)
port_scan['Date'] =  pd.to_datetime(port_scan['ts'], unit='s', utc=True).dt.round('us')

address_scan: pd.DataFrame = pd.read_feather(f'/home/tobias/work/Cloud-Telescope/data/scanner/address_scanner/a_threshold6interval360')
address_scan = test_address_scan
address_scan = address_scan.sort_values(by='ts', ascending=True)
address_scan = address_scan.reset_index(drop=True)
address_scan['Date'] =  pd.to_datetime(address_scan['ts'], unit='s', utc=True).dt.round('us')


mixed_scan_response: MixedScanResponse = calc_mixed_scanner(port_scan=port_scan, address_scan=address_scan)
mixed_scan: pd.DataFrame = mixed_scan_response.mixed_scan
address_scan_unique: pd.DataFrame = mixed_scan_response.address_scan_unique
port_scan_unique: pd.DataFrame = mixed_scan_response.port_scan_unique

In [ ]:
conn: pd.DataFrame = load_latest_dataframe(LogNamesEnum.conn)
conn_udp: pd.DataFrame = conn[conn['proto'] == 'udp']
conn_udp_icmp: pd.DataFrame = conn[conn['proto'].isin(['udp', 'icmp'])]

### save scans as feather

In [ ]:
from src.d01_data.log_names_enum import LogNamesEnum
from src.d01_data.save_files import *
from src.d02_intermediate.add_colums_to_conn import *

def save_scans_as_feather():
    save_dataframe_as_feather(port_scan, LogNamesEnum.df_port_scans, 'init')
    save_dataframe_as_feather(address_scan, LogNamesEnum.df_address_scans, 'init')
    
#save_scans_as_feather()

### calculate heatmap port and address scanner

In [ ]:
from src.d04_analysation.scanning import *

scanning_response: ScanningResponse = calc_total_scanner()
scanner_src_unique_result: ScanningResponse = calc_total_scanner(unique_src=True)
scanner_src_single_result: ScanningResponse = calc_total_scanner(single_src=True)
scanner_mean_duration_result: ScanningResponse = calc_total_scanner(mean_duration=True)

### show heatmap for different interval and threshold 

In [ ]:
import pandas as pd
from src.d03_visiualisation.head_map import *

def show_head_map_duration(df_heatmap: pd.DataFrame, min_threshold: int, max_threshold: int, min_interval: int, max_interval: int, save_name: str, color_scale_description: str):
    df_heatmap["threshold"] = df_heatmap["threshold"].astype(int)
    df_heatmap["interval"] = df_heatmap["interval"].astype(int)
    #df_heatmap["interval"] = df_heatmap["interval"] / 60

    df_heatmap = df_heatmap[df_heatmap['threshold'] >= min_threshold]
    df_heatmap = df_heatmap[df_heatmap['threshold'] <= max_threshold]

    df_heatmap = df_heatmap[df_heatmap['interval'] >= min_interval]
    df_heatmap = df_heatmap[df_heatmap['interval'] <= max_interval]

    show_heat_convert_map_for_df(df=df_heatmap, y_index='threshold', x_columns='interval', value_column='total_scanner', x_label='Intervall', y_label='Anfragen-Grenzwert', save_name= save_name,  color_scale_description=color_scale_description).show()
    

scanning_response.port_scan.loc[scanning_response.port_scan['threshold'] == '4', 'total_scanner'] = 5120



print('scanner_result')
show_head_map_duration(scanning_response.address_scan, min_threshold=4, max_threshold=26, min_interval=60, max_interval=1800, save_name='scanner_result_address', color_scale_description='Anzahl gefundener Scans')
show_head_map_duration(scanning_response.port_scan, min_threshold=4, max_threshold=16, min_interval=60, max_interval=1800, save_name='scanner_result_port', color_scale_description='Anzahl gefundener Scans')


#print('src_unique')
#show_head_map_duration(scanner_src_unique_result.address_scan, min_threshold=4, max_threshold=24, min_interval=60, max_interval=1800)
#show_head_map_duration(scanner_src_unique_result.port_scan, min_threshold=4, max_threshold=12, min_interval=60, max_interval=1800)

print('src_single')
show_head_map_duration(scanner_src_single_result.address_scan, min_threshold=4, max_threshold=26, min_interval=120, max_interval=1800, save_name='src_single_address', color_scale_description='Anzahl neu gefundener Scanner')
show_head_map_duration(scanner_src_single_result.port_scan, min_threshold=4, max_threshold=16, min_interval=120, max_interval=1800, save_name='src_single_port', color_scale_description='Anzahl neu gefundener Scanner')

print('mean_duration')
show_head_map_duration(scanner_mean_duration_result.address_scan, min_threshold=4, max_threshold=26, min_interval=60, max_interval=1800, save_name='duration_address', color_scale_description='durchschnittliche Scandauer')
show_head_map_duration(scanner_mean_duration_result.port_scan, min_threshold=4, max_threshold=16, min_interval=60, max_interval=1800, save_name='duration_port', color_scale_description='durchschnittliche Scandauer')

### show multi graf interval threshold

In [ ]:
from src.d03_visiualisation.graf_interval_threshold import *
from sklearn.preprocessing import MinMaxScaler

def show_multi_graf_interval_threshold(threshold: int, df_graf_1: pd.DataFrame, df_graf_2: pd.DataFrame, df_graf_3: pd.DataFrame, saving_name: str):
    scaler = MinMaxScaler()

    graf_scanning_response: pd.DataFrame = df_graf_1[df_graf_1['threshold'] == threshold]
    graf_scanning_response['total_scanner'] = graf_scanning_response['total_scanner'].fillna(0)
    graf_scanning_response['total_scanner'] = graf_scanning_response['total_scanner'].astype('float')
    graf_scanning_response["scaled_scanner"] = scaler.fit_transform(graf_scanning_response[["total_scanner"]])

    graf_src_single: pd.DataFrame = df_graf_2[df_graf_2['threshold'] == threshold]
    graf_src_single['total_scanner'] = graf_src_single['total_scanner'].fillna(0)
    graf_src_single['total_scanner'] = graf_src_single['total_scanner'].astype('float')
    graf_src_single["scaled_scanner"] = scaler.fit_transform(graf_src_single[["total_scanner"]])


    graf_mean_duration: pd.DataFrame = df_graf_3[df_graf_3['threshold'] == threshold]
    graf_mean_duration['total_scanner'] = graf_mean_duration['total_scanner'].fillna(0)
    graf_mean_duration['total_scanner'] = graf_mean_duration['total_scanner'].astype('float')
    graf_mean_duration["scaled_scanner"] = scaler.fit_transform(graf_mean_duration[["total_scanner"]])


    df_list: list[pd.DataFrame] = [graf_scanning_response, graf_src_single, graf_mean_duration]
    df_labels: list[str] = ['Scans', 'neue Scanner', 'Scan-Dauer']
    df_linestyle: list[str] = ['-', '--', '-.']
    df_marker: list[str] = ['o', 's', '^']
    graf_new_interval_threshold(dataframes=df_list, df_labels=df_labels, df_linestyle=df_linestyle, df_marker=df_marker, x_label='Intervall', y_label='normalisierter Funktionswert', saving_name=saving_name).show()

df_graf_scanning_response: pd.DataFrame = scanning_response.address_scan[scanning_response.address_scan['interval'] < 700]
src_single_scanning_response: pd.DataFrame = scanner_src_single_result.address_scan[scanner_src_single_result.address_scan['interval'] < 700]
mean_duration_scanning_response: pd.DataFrame = scanner_mean_duration_result.address_scan[scanner_mean_duration_result.address_scan['interval'] < 700]

df_graf_scanning_port: pd.DataFrame = scanning_response.port_scan[scanning_response.port_scan['interval'] < 700]
src_single_scanning_port: pd.DataFrame = scanner_src_single_result.port_scan[scanner_src_single_result.port_scan['interval'] < 700]
mean_duration_scanning_port: pd.DataFrame = scanner_mean_duration_result.port_scan[scanner_mean_duration_result.port_scan['interval'] < 700]

print(f'address')
show_multi_graf_interval_threshold(threshold=6, df_graf_1=df_graf_scanning_response, df_graf_2=src_single_scanning_response, df_graf_3=mean_duration_scanning_port, saving_name='address_graf_multi')

print(f'port')
show_multi_graf_interval_threshold(threshold=6, df_graf_1=df_graf_scanning_port, df_graf_2=src_single_scanning_port, df_graf_3=mean_duration_scanning_response, saving_name='port_graf_multi')


### show percent distribution of different hosts mapped address scan

In [ ]:
from src.d03_visiualisation.bar_diagram import *

def show_percent_address_scanner2():
    address_df: pd.DataFrame = scanning_response.address_scan
    address_df = address_df[address_df['interval'] == '2'].copy()
    address_df["threshold"] = address_df["threshold"].astype(int)
    address_df = address_df[address_df['threshold'] >= 20]
    
    
    basic_bar_diagram(address_df, 'threshold', 'total_scanner', 'Anzahl Scanner', 'min. Anzahl unterschiedlicher Ziel-IP').show()
    
    return address_df


def show_percent_address_scanner():
    merged_df: pd.DataFrame = calculate_proportion_address_port( scanning_response=scanning_response, interval=240,min_threshold_address=4,max_threshold_address= 50, min_threshold_port=4, max_threshold_port=50)

    y_columns: list[str] = ['total_scanner_address', 'total_scanner_port']
    y_description: list[str] = ['Address Scanner', 'Port Scanner']
    multi_bar_diagram(df=merged_df, x_column='threshold', y_columns=y_columns, x_label="min. Anzahl unterschiedlicher Ziel-IP's / Ports", y_label='Anzahl Scanner', y_descriptions=y_description).show()
    
show_percent_address_scanner()

### print scanner host IP for CAIDA script

In [ ]:

def print_caida_ips():
    output: str = ''
    founded_ips: list[str] = []
    for index, ip in enumerate(port_unique['src'].tolist()):
        founded_ips.append(ip)
        if index == 0:
            output = f'host {ip}'
        else:
            output = f'{output} || {ip}'
    
    for ip in address_unique['src'].tolist():
        if ip not in founded_ips:
            founded_ips.append(ip)
            output = f'{output} || {ip}'
    
    print(f'Length: {len(founded_ips)}')
    
    print(output)
    
#print_caida_ips()

### show port, address and mixed scans mapped to countries

In [ ]:
from src.d04_analysation.scnas_mapped_to_country import *
from src.d03_visiualisation.bar_diagram import *
import pandas as pd
import matplotlib.pyplot as plt

def show_scans_mapped_countries():
    mapped_result: pd.DataFrame = scans_mapped_country(conn_upd=conn_udp, address_df=address_scan, port_df=port_scan, mixed_df=mixed_scan)
    
    mapped_result = mapped_result.head(20)
    
    stacked_three_bar_diagram(df=mapped_result, x_column='orig_country_location', y1_column='address_scan', y2_column='port_scan', y3_column='mixed_scan', x_lable='Länderkürzel', y_lable='Anzahl Scans', y1_description='Address-Scan', y2_description='Port-Scan', y3_description='Mixed-Scan').show()
    
    return mapped_result

mapped_result_country: pd.DataFrame = show_scans_mapped_countries()

### show aggressive mapped to counties

In [ ]:
from src.d04_analysation.scan_aggrssivity_mapped_countries import *
from src.d03_visiualisation.bar_diagram import *
def show_aggressive_mapped_countries():
    aggressivity_mapped: pd.DataFrame = aggressivity_mapped_countries(mapped_result_country, conn_udp, address_scan, port_scan)
    
    multi_bar_diagram(df=aggressivity_mapped, x_column='country', y_columns=['port_aggressivity', 'address_aggressivity'], x_lable='Land', y_lable='Durchschnitt gescannter Hosts/Ports', y_descriptions=['Port Scan', 'Address Scan'], bar_width=0.4, rotation=0).show()
    
show_aggressive_mapped_countries()

### show percent distribution of address scan mapped to protocol 

In [ ]:
from src.d04_analysation.count_percent import *
from src.d03_visiualisation.bar_diagram import *

udp_services = {
    53: "DNS",
    5060: "SIP",
    123: "NTP",
    5683: "CoAP",
    3702: "WS-Discovery",
    161: "SNMP",
    37020: "WS-Discovery (Secure)",
    389: "LDAP",
    1900: "SSDP",
    33434: "Traceroute",
    1194: "OpenVPN",
    3283: "Apple Remote Desktop",
    3478: "STUN",
    9034: "Tor ORPort",
    500: "ISAKMP (IPsec)",
    137: "NetBIOS Name Service",
    27015: "Source Engine",
    111: "ONC RPC",
    27960: "Quake 3",
    19: "Chargen",
}

def show_address_mapped_to_protocol():
    port_calculation: list[ColumnResults] = column_calculation(address_scan, {'p': -1}, False)
    df_address: pd.DataFrame = port_calculation[0].result_as_df 
    df_address = df_address.drop(index=0)
    df_address = df_address.head(20)
    df_address = df_address.reset_index(drop=True)

    for index, port in enumerate(df_address['p'].tolist()):
        service = udp_services.get(int(port))
        df_address.loc[index, 'service'] = service

    seaborn_bar2_diagram(df=df_address, x_column='service', y_column='percent', x_lable='Protokoll', y_lable='Prozentwerte', rotation=40, saving_name='protocol', set_position=10.2)
    
    return df_address

show_address_mapped_to_protocol()

### show total distribution of address scan mapped to biggist countries 

In [ ]:
from src.d04_analysation.scans_proto_country import *

def show_address_scans_port_mapped_countries():
    port_calculation: list[ColumnResults] = column_calculation(address_scan, {'p': -1}, False)

    result_df: pd.DataFrame = calculate_scans_proto_country(conn_upd=conn_udp, address_scan=address_scan, mapped_country=mapped_result_country, mapped_ports=port_calculation[0].result_as_df)

    for index, port in enumerate(result_df['port'].tolist()):
        service = udp_services.get(int(port))
        result_df.loc[index, 'service'] = service

    y_columns: list[str] = [result_df.columns[2], result_df.columns[3], result_df.columns[4], result_df.columns[5]]



    multi_bar_diagram(df=result_df, x_column='service', y_columns=y_columns, x_label='Port', y_label='Scanner', y_descriptions=y_columns, bar_width=0.2, rotation=45).show()

    multi_color_new2_bar_diagram(df=result_df, x_column='service', y_columns=y_columns, x_label='Protokoll', y_label='Anzahl Scans', y_descriptions=y_columns, bar_width=0.2, rotation=45)
    
show_address_scans_port_mapped_countries()

### calculate arrival rate for address and port

In [ ]:
from src.d04_analysation.arrival_rate import *
from datetime import datetime

def calculate_arrival_rate(arrival_df: pd.DataFrame, start: datetime, end: datetime, time_dimension: DateTimeEnum, timestamp_parsing: bool):
    
    df_scaner_arrival: pd.DataFrame = arrival_df.sort_values(by='ts', ascending=True, inplace=False)
    df_scaner_arrival = df_scaner_arrival[['ts']]
    
    if timestamp_parsing:
        df_scaner_arrival["ts"] = pd.to_datetime(df_scaner_arrival["ts"], unit='s', utc=True)
        df_scaner_arrival["ts"] = df_scaner_arrival["ts"].dt.floor("ms")
        
    df_scan_arrival: ArrivalRateReturn = get_arrival_rate(df_scaner_arrival, start, end, time_dimension)

    print(f'From {start} (Berlin) to {end} (Berlin) the telescope captured: {df_scan_arrival.mean_arrival_rate} connections per {time_dimension.value}')

    return df_scan_arrival.arrival_rate_df

specific_start = datetime(2025, 1, 20, 15, 58, tzinfo=timezone.utc)
specific_end = datetime(2025, 2, 3, 1, 33, tzinfo=timezone.utc)

calc_arrival_address: pd.DataFrame = calculate_arrival_rate(address_scan, specific_start, specific_end, DateTimeEnum.minute, True)

calc_arrival_port: pd.DataFrame = calculate_arrival_rate(port_scan, specific_start, specific_end, DateTimeEnum.minute, True)

calc_total_port_traffic_raw: pd.DataFrame = conn_udp[conn_udp['id.orig_h'].isin(port_scan['src'])]
calc_total_port_traffic = calculate_arrival_rate(calc_total_port_traffic_raw, specific_start, specific_end, DateTimeEnum.minute, True)

calc_total_address_traffic_raw: pd.DataFrame = conn_udp[conn_udp['id.orig_h'].isin(address_scan['src'])]
calc_total_address_traffic = calculate_arrival_rate(calc_total_address_traffic_raw, specific_start, specific_end, DateTimeEnum.minute, True)

### show arrival rate for address and port

In [ ]:
from src.d03_visiualisation.graf_arrival_rate import *
from src.d04_analysation.arrival_rate import *

def show_arrival_rate(arrival_df: pd.DataFrame):

    graf_arrival_rate_days(arrival_df).show()

block_size_arrival = 60
block_size_traffic = 60


calc_arrival_port = calc_arrival_port.reset_index(drop=True)
calc_arrival_address = calc_arrival_address.reset_index(drop=True)
calc_total_port_traffic = calc_total_port_traffic.reset_index(drop=True)
calc_total_address_traffic = calc_total_address_traffic.reset_index(drop=True)

calc_arrival_port_aggregated = (
    calc_arrival_port.groupby(calc_arrival_port.index // block_size_arrival)
    .agg(timestamp=("timestamp", "first"), total_count=("total_count", "sum"))
    .reset_index(drop=True)
)
calc_arrival_address_aggregated = (
    calc_arrival_address.groupby(calc_arrival_address.index // block_size_arrival)
    .agg(timestamp=("timestamp", "first"), total_count=("total_count", "sum"))
    .reset_index(drop=True)
)

calc_total_port_traffic_aggregated = (
    calc_total_port_traffic.groupby(calc_total_port_traffic.index // block_size_traffic)
    .agg(timestamp=("timestamp", "first"), total_count=("total_count", "sum"))
    .reset_index(drop=True)
)

calc_total_address_traffic_aggregated = (
    calc_total_address_traffic.groupby(calc_total_address_traffic.index // block_size_traffic)
    .agg(timestamp=("timestamp", "first"), total_count=("total_count", "sum"))
    .reset_index(drop=True)
)

#show_arrival_rate(calc_arrival_port_aggregated)
#show_arrival_rate(calc_arrival_address_aggregated)
#show_arrival_rate(calc_total_port_traffic_aggregated)
#show_arrival_rate(calc_total_address_traffic_aggregated)

arrival_df_list: list[pd.DataFrame] = [calc_arrival_port_aggregated, calc_arrival_address_aggregated]
total_traffic_df_list: list[pd.DataFrame] = [calc_total_port_traffic_aggregated, calc_total_address_traffic_aggregated]
multi_arrival_rate_days(dfs=arrival_df_list, df_colors=['g', 'b'], df_descriptions=['Port Scan', 'Address Scan'], x_description='Tage', y_description='Anzahl Port-/Address-Scans', y_ticks=[0, 4, 8 , 12, 16, 20, 40, 80, 160, 320], linthresh=20, grade='symlog', saving_name='address2')
multi_arrival_rate_days(dfs=total_traffic_df_list, df_colors=['g', 'b'], df_descriptions=['Port Scan', 'Address Scan'], x_description='Tage', y_description='Anzahl Anfragen Port-/Address-Scans', y_ticks=[0, 150, 300, 450, 600, 750, 900, 1800, 3600, 7200, 14400], linthresh=900, grade='symlog', saving_name='port2')

# total scans::
# 1. ssd networks limited 2025-01-24 19:58:00+00:00 - 2025-01-24 20:58:00+00:00 
# 2. Mobile Communication Company of Iran PLC  Iran Telecommunication Company PJS  _ 2025-01-26 09:58:00+00:00 - 2025-01-26 15:58:00+00:00
# 3. Information Technology Company (ITC) _ Saudi Telecom Company JSC _ Mobile Communication Company of Iran PLC 14% 2025-01-30 09:58:00+00:00 - 2025-01-30 13:58:00+00:00

# traffic
# 1 gcommer.com 2025-01-28 18:58:00+00:00 - 2025-01-26 15:58:00+00:00 
# 2. Mobile Communication Company of Iran PLC  Iran Telecommunication Company PJS  2025-01-26 09:58:00+00:00 - 2025-01-26 15:58:00+00:00
# 3. Perfecto Consultoria E Apoio Administrativo LTDA  2025-01-26 2025-01-26 09:58:00+00:00 - 2025-01-26 15:58:00+00:00
# 4 address Perfecto Consultoria E Apoio Administrativo LTDA,  LLC 188.166.250.148 ssd networks limited 2025-01-24 19:58:00+00:00 - 2025-01-24 20:58:00+00:00
# 5 port ssd networks limited 92% 19:58:00+00:00 - 2025-01-24 20:58:00+00:00
# 6 address Information Technology Company (ITC) Aryan Satellite Co. (Private Joint Stock) Mobile Communication Company of Iran PLC  Saudi Telecom Company JSC 11 2025-01-26 09:58:00+00:00 - 2025-01-26 15:58:00+00:00
# 7 Pfcloud UG 16 Optibounce, LLC 10 2025-01-26 09:58:00+00:00 - 2025-01-26 15:58:00+00:00


In [ ]:
d1_start_1 = pd.to_datetime('2025-01-24 19:58:00+00:00')
d1_end_1 = pd.to_datetime('2025-01-24 21:58:00+00:00')
d1_start_2 = pd.to_datetime('2025-01-26 09:58:00+00:00')
d1_end_2 = pd.to_datetime('2025-01-26 16:58:00+00:00')
d1_start_3 = pd.to_datetime('2025-01-30 09:58:00+00:00')
d1_end_3 = pd.to_datetime('2025-01-30 13:58:00+00:00')

d1_address = address_scan
d1_port = port_scan
d1_address['ts'] = pd.to_datetime(d1_address['ts'], unit='s', utc=True)
d1_port['ts'] = pd.to_datetime(d1_port['ts'], unit='s', utc=True)

# d1_1_outlier
d1_1_outlier = d1_address[d1_address['ts'] >= d1_start_1]
d1_1_outlier = d1_1_outlier[d1_1_outlier['ts'] <= d1_end_1]
d1_1_outlier_src: list[ColumnResults] = column_calculation(d1_1_outlier, {'src': -1}, False)[0].result_as_df

# d1_2_outlier
d1_2_outlier = d1_port[d1_port['ts'] >= d1_start_2]
d1_2_outlier = d1_2_outlier[d1_2_outlier['ts'] <= d1_end_2]
d1_2_outlier_conn = conn_udp[conn_udp['id.orig_h'].isin(d1_2_outlier['src'])]
d1_2_outlier_conn_rir: list[ColumnResults] = column_calculation(d1_2_outlier_conn, {'orig_as_name_rir': -1}, False)[0].result_as_df

# d1_3_outlier
d1_3_outlier = d1_port[d1_port['ts'] >= d1_start_3]
d1_3_outlier = d1_3_outlier[d1_3_outlier['ts'] <= d1_end_3]
d1_3_outlier_conn = conn_udp[conn_udp['id.orig_h'].isin(d1_3_outlier['src'])]
d1_3_outlier_conn_rir: list[ColumnResults] = column_calculation(d1_3_outlier_conn, {'orig_as_name_rir': -1}, False)[0].result_as_df

In [ ]:
d2_start_1 = pd.to_datetime('2025-01-28 18:58:00+00:00')
d2_end_1 = pd.to_datetime('2025-01-29 02:58:00+00:00')
d2_start_2 = pd.to_datetime('2025-01-26 09:58:00+00:00')
d2_end_2 = pd.to_datetime('2025-01-26 16:58:00+00:00')
d2_start_3 = pd.to_datetime('2025-01-26 09:58:00+00:00')
d2_end_3 = pd.to_datetime('2025-01-26 16:58:00+00:00')
d2_start_4 = pd.to_datetime('2025-01-24 19:58:00+00:00')
d2_end_4 = pd.to_datetime('2025-01-24 21:58:00+00:000')
d2_start_5 = pd.to_datetime('2025-01-24 19:58:00+00:00')
d2_end_5 = pd.to_datetime('2025-01-24 21:58:00+00:000')
d2_start_6 = pd.to_datetime('2025-01-30 09:58:00+00:00')
d2_end_6 = pd.to_datetime('2025-01-30 13:58:00+00:00')
d2_start_7 = pd.to_datetime('2025-01-30 09:58:00+00:00')
d2_end_7 = pd.to_datetime('2025-01-30 13:58:00+00:00')
d2_start_8 = pd.to_datetime('2025-01-25 01:58:00+00:00')
d2_end_8 = pd.to_datetime('2025-01-25 08:58:00+00:00')

d2_address = calc_total_address_traffic_raw
d2_port = calc_total_port_traffic_raw
d2_address['ts'] = pd.to_datetime(d2_address['ts'], unit='s', utc=True)
d2_port['ts'] = pd.to_datetime(d2_port['ts'], unit='s', utc=True)

# d2_1_outlier
d2_1_outlier = d2_port[d2_port['ts'] >= d2_start_1]
d2_1_outlier = d2_1_outlier[d2_1_outlier['ts'] <= d2_end_1]
d2_1_outlier_src: list[ColumnResults] = column_calculation(d2_1_outlier, {'id.orig_h': -1}, False)[0].result_as_df

# d2_2_outlier
d2_2_outlier = d2_port[d2_port['ts'] >= d2_start_2]
d2_2_outlier = d2_2_outlier[d2_2_outlier['ts'] <= d2_end_2]
d2_2_outlier_conn = conn_udp[conn_udp['id.orig_h'].isin(d2_2_outlier['id.orig_h'])]
d2_2_outlier_conn_rir: list[ColumnResults] = column_calculation(d2_2_outlier_conn, {'orig_as_name_rir': -1}, False)[0].result_as_df

# d2_3_outlier
d2_3_outlier = d2_address[d2_address['ts'] >= d2_start_3]
d2_3_outlier = d2_3_outlier[d2_3_outlier['ts'] <= d2_end_3]
d2_3_outlier_conn = conn_udp[conn_udp['id.orig_h'].isin(d2_3_outlier['id.orig_h'])]
d2_3_outlier_conn_rir: list[ColumnResults] = column_calculation(d2_3_outlier_conn, {'orig_as_name_rir': -1}, False)[0].result_as_df

# d2_4_outlier
d2_4_outlier = d2_address[d2_address['ts'] >= d2_start_4]
d2_4_outlier = d2_4_outlier[d2_4_outlier['ts'] <= d2_end_4]
d2_4_outlier_conn = conn_udp[conn_udp['id.orig_h'].isin(d2_4_outlier['id.orig_h'])]
d2_4_outlier_conn_rir: list[ColumnResults] = column_calculation(d2_4_outlier_conn, {'orig_as_name_rir': -1}, False)[0].result_as_df

# d2_4_outlier
d2_5_outlier = d2_port[d2_port['ts'] >= d2_start_5]
d2_5_outlier = d2_5_outlier[d2_5_outlier['ts'] <= d2_end_5]
d2_5_outlier_conn = conn_udp[conn_udp['id.orig_h'].isin(d2_5_outlier['id.orig_h'])]
d2_5_outlier_conn_rir: list[ColumnResults] = column_calculation(d2_5_outlier_conn, {'orig_as_name_rir': -1}, False)[0].result_as_df

# d2_4_outlier
d2_6_outlier = d2_port[d2_port['ts'] >= d2_start_6]
d2_6_outlier = d2_6_outlier[d2_6_outlier['ts'] <= d2_end_6]
d2_6_outlier_conn = conn_udp[conn_udp['id.orig_h'].isin(d2_6_outlier['id.orig_h'])]
d2_6_outlier_conn_rir: list[ColumnResults] = column_calculation(d2_6_outlier_conn, {'orig_as_name_rir': -1}, False)[0].result_as_df

# d2_4_outlier
d2_7_outlier = d2_address[d2_address['ts'] >= d2_start_7]
d2_7_outlier = d2_7_outlier[d2_7_outlier['ts'] <= d2_end_7]
d2_7_outlier_conn = conn_udp[conn_udp['id.orig_h'].isin(d2_7_outlier['id.orig_h'])]
d2_7_outlier_conn_rir: list[ColumnResults] = column_calculation(d2_7_outlier_conn, {'orig_as_name_rir': -1}, False)[0].result_as_df

d2_8_outlier = d2_address[d2_address['ts'] >= d2_start_8]
d2_8_outlier = d2_8_outlier[d2_8_outlier['ts'] <= d2_end_8]
d2_8_outlier_conn = conn_udp[conn_udp['id.orig_h'].isin(d2_8_outlier['id.orig_h'])]
d2_8_outlier_conn_rir: list[ColumnResults] = column_calculation(d2_8_outlier_conn, {'orig_as_name_rir': -1}, False)[0].result_as_df

### show PIR for top 20 scanner

In [ ]:

try:
    import dns
except ModuleNotFoundError:
    import sys
    !{sys.executable} -m pip install dnspython
    import dns
try:
    import brokenaxes
except ModuleNotFoundError:
    import sys
    import subprocess
    subprocess.check_call([sys.executable, "-m", "pip", "install", "brokenaxes"])
    import brokenaxes
from src.d04_analysation.count_percent import *
from src.d04_analysation.reverse_dns_lookup import *
from src.d04_analysation.find_scanner import *
from src.d03_visiualisation.bar_diagram import *

def calc_and_show_top_scanner():
    top_df_response: FindScannerReturn = calc_top_scanner(port_scan=port_scan, address_scan=address_scan, conn_udp=conn_udp)
    


    address_custom_ticks_ax4 = [0, 2, 4 , 6, 8, 10, 20, 40, 80, 160, 320]
    address_custom_ticks_ax2 = [0, 2000, 4000 , 6000, 8000, 10000, 20000, 40000, 80000]
    address_custom_ticks_ax3 = [0, 300, 10, 15, 20, 25, 30, 40, 45]

    mask = top_df_response.address_scanner['orig_as_name_rir'] == 'Perfecto Consultoria E Apoio Administrativo LTDA'

    top_df_response.address_scanner.loc[mask, 'orig_as_name_rir'] = 'Perfecto Consultoria E Apoio \n Administrativo LTDA'
    address_multi_new_3_color_bar_diagram(df=top_df_response.address_scanner, x_column='orig_as_name_rir', y_columns=['total_scanner', 'total_connections', 'mean_duration', 'unique_ports_host'], x_label='Source IP', y_label='Anzahl', y_descriptions=['Anzahl Scans', 'Anzahl Anfragen', 'Scan Dauer', "Anzahl unterschiedlicher Ziel-Ports"], bar_width=0.2, rotation=20, address_custom_ticks_ax2=address_custom_ticks_ax2,address_custom_ticks_ax4=address_custom_ticks_ax4).show()

    port_custom_ticks_ax3 = [0, 0.1, 0.2 ,0.3, 0.4, 0.5, 1, 2, 4, 8, 15, 30, 60, 120, 240, 480, 960, 1920]
    port_custom_ticks_ax2 = [0, 200, 400, 600 , 800, 1000, 2000, 4000, 8000, 16000]
    custom_ticks_ax4 = [0, 5, 10, 15, 20, 25, 30, 35,40, 45, 50, 100, 200]
    port_multi_new_color_bar_diagram(df=top_df_response.port_scanner, x_column='orig_as_name_rir', y_columns=['total_scanner', 'total_connections', 'mean_duration', 'unique_ports_host'], x_label='Source IP', y_label='Anzahl', y_descriptions=['Anzahl Scans', 'Anzahl Anfragen', 'Scan Dauer', "Anzahl unterschiedlicher Ziel-IPs"], bar_width=0.2, rotation=20, custom_ticks_ax3=port_custom_ticks_ax3, custom_ticks_ax2=port_custom_ticks_ax2, custom_ticks_ax4=custom_ticks_ax4).show()
    #
    return top_df_response.port_scanner



calc_address_ = calc_and_show_top_scanner()

### show scanner type

In [ ]:
from src.d03_visiualisation.bar_diagram import *
from src.d04_analysation.count_percent import *

def show_attacked_ports():
    
    attack_ports_calculation: pd.DataFrame = column_calculation(backscatter_traffic_unique_attack, {'id.resp_p': -1}, False)[0].result_as_df
    attack_ports_calculation = attack_ports_calculation.drop(index=attack_ports_calculation.index[0])
    basic_bar_new_2_diagram(df=attack_ports_calculation, x_column='id.resp_p', y_column='total', x_label='Port', y_label='Anzahl Angriffe', rotation=40, saving_name='ddos_ports', width=1, bar_spacing=0.2, diagramm_width=5, diagramm_height=5).show()
    
    return attack_ports_calculation
    
show_attacked_ports()


### show education table

In [ ]:
from src.d04_analysation.scan_education import *
from src.d03_visiualisation.tabel_visiualisation import *
def show_education_table():
    education_df: pd.DataFrame = calc_scan_education(conn_udp)
    
    show_as_basic_table(education_df).show()
    return education_df
    
    
show_education_table()

In [ ]:
from src.d04_analysation.scan_education import *
from src.d03_visiualisation.tabel_visiualisation import *
def show_education_table():
    education_df: pd.DataFrame = calc_scan_education(conn_udp)
    education_df.rename(columns={"DNS Eintrag": "Dns_Eintrag"}, inplace=True)
    education_df.rename(columns={"RIR Name": "RIR_Name"}, inplace=True)
    education_df.rename(columns={"Anzahl IP\'s": "Anzahl_IPs"}, inplace=True)
    education_df.rename(columns={"Universität": "Forschungseinrichtung"}, inplace=True)
    education_df['Dns_Eintrag'] = education_df['Dns_Eintrag'].fillna("")
    education_df['RIR_Name'] = education_df['RIR_Name'].fillna("")
    education_df["Anzahl_IPs"] = education_df['Anzahl_IPs'].fillna("")
    
    
    

    column_widths = {
        "Forschungseinrichtung": 0.67,
        "Land": 0.15,
        "Dns_Eintrag": 0.8,
        "Ports": 0.3,
        "RIR_Name": 0.7
    }

    col_left_offsets = {
        "Forschungseinrichtung": 0.00001,
        "Land": 0.03,
        "Dns_Eintrag": 0.03,
        "Ports": 0.93,
        "RIR_Name": 0.07
    }

    row_heights = [0.5] + [0.3, 0.4, 0.6, 0.5, 0.7, 0.4, 0.6, 0.3]  #
    row_heights = [0.08, 0.16, 0.1, 0.1, 0.16, 0.24, 0.1, 0.1, 0.24, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1] 
    
    show_as_basic2_table(education_df, column_widths, row_heights, col_left_offsets)
    plt.show()
    return education_df
    
    
show_education_table()

### test Data for find_scanner 

In [ ]:
import datetime
from src.d04_analysation.find_scanner import *
scanner_test_df: pd.DataFrame = pd.DataFrame({
    'ts': pd.Series(dtype='float'),
    'id.orig_h': pd.Series(dtype='str'),
    'id.orig_p': pd.Series(dtype='str'),
    'id.resp_h': pd.Series(dtype='str'),
    'id.resp_p': pd.Series(dtype='str'),
})

for i in range(10, 38):
    new_row: pd.DataFrame = pd.DataFrame({
        'ts': [datetime.datetime.fromisoformat(f"2024-06-01T12:00:{i}").timestamp()],
        'id.orig_h': [f'5.252.176.74'],
        'id.orig_p': ['53'],
        'id.resp_h': [f'170.64.134.{i}'],
        'id.resp_p': ['98'],
    })
    scanner_test_df = pd.concat([scanner_test_df, new_row], ignore_index=True)

new_row_2: pd.DataFrame = pd.DataFrame({
        'ts': [datetime.datetime.fromisoformat(f"2024-06-01T12:02:45").timestamp()],
        'id.orig_h': [f'5.252.176.74'],
        'id.orig_p': ['53'],
        'id.resp_h': [f'170.64.134.99'],
        'id.resp_p': ['98'],
    })
scanner_test_df = pd.concat([scanner_test_df, new_row_2], ignore_index=True)

# for i in range(10, 38):
#     new_row: pd.DataFrame = pd.DataFrame({
#         'ts': [datetime.datetime.fromisoformat(f"2024-06-01T12:07:{i}").timestamp()],
#         'id.orig_h': [f'5.252.176.74'],
#         'id.orig_p': ['53'],
#         'id.resp_h': [f'170.64.134.{i}'],
#         'id.resp_p': ['11'],
#     })
#     scanner_test_df = pd.concat([scanner_test_df, new_row], ignore_index=True)
#
# scanner_test_df = pd.concat([scanner_test_df, new_row_2], ignore_index=True)


for i in range(10, 38):
    new_row: pd.DataFrame = pd.DataFrame({
        'ts': [datetime.datetime.fromisoformat(f"2024-06-01T12:00:{i}").timestamp()],
        'id.orig_h': [f'5.252.176.74'],
        'id.orig_p': ['53'],
        'id.resp_h': [f'170.64.134.10'],
        'id.resp_p': [f'{i}'],
    })
    scanner_test_df = pd.concat([scanner_test_df, new_row], ignore_index=True)

new_row_2: pd.DataFrame = pd.DataFrame({
        'ts': [datetime.datetime.fromisoformat(f"2024-06-01T12:01:38").timestamp()],
        'id.orig_h': [f'5.252.176.74'],
        'id.orig_p': ['53'],
        'id.resp_h': [f'170.64.134.10'],
        'id.resp_p': ['98'],
    })
scanner_test_df = pd.concat([scanner_test_df, new_row_2], ignore_index=True)


test_address_unique: pd.DataFrame = pd.DataFrame({
    'src': [f'5.252.176.74'],
})
test_port_unique: pd.DataFrame = pd.DataFrame({
    'src': [f'5.252.176.74'],
})
find_scanner_return: FindScannerReturn = find_scanning(unique_port_scanner=test_port_unique, unique_address_scanner=test_address_unique, conn_udp=scanner_test_df, threshold=29, interval=3)


display(find_scanner_return.address_scanner)
find_scanner_return.port_scanner
#scanner_test_df

